In [52]:
account_id_map = {
    'us-east-1': '785573368785',
    'us-east-2': '007439368137',
    'us-west-1': '710691900526',
    'us-west-2': '301217895009',
    'eu-west-1': '802834080501',
    'eu-west-2': '205493899709',
    'eu-west-3': '254080097072',
    'eu-north-1': '601324751636',
    'eu-south-1': '966458181534',
    'eu-central-1': '746233611703',
    'ap-east-1': '110948597952',
    'ap-south-1': '763008648453',
    'ap-northeast-1': '941853720454',
    'ap-northeast-2': '151534178276',
    'ap-southeast-1': '324986816169',
    'ap-southeast-2': '355873309152',
    'cn-northwest-1': '474822919863',
    'cn-north-1': '472730292857',
    'sa-east-1': '756306329178',
    'ca-central-1': '464438896020',
    'me-south-1': '836785723513',
    'af-south-1': '774647643957'
}

In [53]:
!pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com


In [55]:
!pip install nvidia-pyindex
!pip install tritonclient[http]

!pip install -qU pip awscli boto3 sagemaker transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com, https://pypi.ngc.nvidia.com
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to

**Test how to create BERT torchscript model**

In [82]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GPTJForCausalLM, AutoTokenizer
from transformers import GPT2Tokenizer
from transformers import BertModel, BertTokenizer, BertConfig
import torch

prompt = (

    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "

    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "

    "researchers was the fact that the unicorns spoke perfect English."

)
# Create the GPTJ Tokenzier
tokenizer_gptj = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B" ,device_map="auto", load_in_8bit=True)
#tokenizer_gptj = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-j-6B")

# Create the Model GPT j 
#model_gptj = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", device_map="auto", load_in_8bit=True)
#model_gptj = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B") #,torch_dtype=torch.float16)  - you need to have a machine which implements float16

# This does not download if you specify float16 or 32
model_gptj = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B",torch_dtype=torch.float,low_cpu_mem_usage=True)

# Using RAW BERT models with no head  -- tokenizer does not work 
#model_gptj = BertModel.from_pretrained("EleutherAI/gpt-j-6B", torchscript=True, torch_dtype=torch.float,)
#tokenizer_gptj = BertTokenizer.from_pretrained("EleutherAI/gpt-j-6B")


In [83]:
model_gptj

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
  

In [84]:
tokenizer_gptj

PreTrainedTokenizerFast(name_or_path='EleutherAI/gpt-j-6B', vocab_size=50257, model_max_len=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)})

In [85]:
print(tokenizer_gptj(prompt).input_ids)
print(len(tokenizer_gptj(prompt).input_ids))
print(tokenizer_gptj(prompt, return_tensors="pt").input_ids)
print(len(tokenizer_gptj(prompt, return_tensors="pt").input_ids))

[818, 257, 14702, 4917, 11, 5519, 5071, 257, 27638, 286, 28000, 19942, 2877, 287, 257, 6569, 11, 4271, 31286, 1850, 19272, 11, 287, 262, 843, 274, 21124, 13, 3412, 517, 6452, 284, 262, 4837, 373, 262, 1109, 326, 262, 28000, 19942, 5158, 2818, 3594, 13]
45
tensor([[  818,   257, 14702,  4917,    11,  5519,  5071,   257, 27638,   286,
         28000, 19942,  2877,   287,   257,  6569,    11,  4271, 31286,  1850,
         19272,    11,   287,   262,   843,   274, 21124,    13,  3412,   517,
          6452,   284,   262,  4837,   373,   262,  1109,   326,   262, 28000,
         19942,  5158,  2818,  3594,    13]])
1


In [15]:
test_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
test_tokenizer(prompt, return_tensors="pt").input_ids

tensor([[  818,   257, 14702,  4917,    11,  5519,  5071,   257, 27638,   286,
         28000, 19942,  2877,   287,   257,  6569,    11,  4271, 31286,  1850,
         19272,    11,   287,   262,   843,   274, 21124,    13,  3412,   517,
          6452,   284,   262,  4837,   373,   262,  1109,   326,   262, 28000,
         19942,  5158,  2818,  3594,    13]])

In [86]:
# Predict using the model created from GPTJ Config
gen_tokens = model_gptj.generate(

    input_ids = tokenizer_gptj(prompt, return_tensors="pt").input_ids,

    do_sample=True,

    temperature=0.9,

    max_length=100,

)
gen_tokens

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[  818,   257, 14702,  4917,    11,  5519,  5071,   257, 27638,   286,
         28000, 19942,  2877,   287,   257,  6569,    11,  4271, 31286,  1850,
         19272,    11,   287,   262,   843,   274, 21124,    13,  3412,   517,
          6452,   284,   262,  4837,   373,   262,  1109,   326,   262, 28000,
         19942,  5158,  2818,  3594,    13,  1119,   547,   588,  3487,    11,
          1692,    12, 11534,  4695,    11,  2845,   329,   262, 12718,   319,
           511, 22645,    13,  1119,   635,  3114,   655,   588,  3218,  4695,
            11,   475,   351,  1263, 26771,   326,  6348,   319,   511,  1674,
         16600,    13,   198,   198,   464,  5519,  5071,   262,  6283,   649,
          4693,   832,  7446,  5254,    13,  1119,  4966,   262,  2910,   286]])

In [87]:
tokenizer_gptj.batch_decode(gen_tokens)[0]

'In a shocking finding, scientists discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. They were like normal, human-looking animals, except for the horn on their forehead. They also looked just like regular animals, but with big horns that grew on their foreheads.\n\nThe scientists discovered the strange new species through DNA tests. They ran the blood of'

### Create the BERT Model in Torch Script mode -- .pt model
use the pre trained and use torchscript flag here

### BELOW is not needed

In [90]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

bs = 1
seq_len = 512
dummy_inputs = [
    torch.randint(1000, (bs, seq_len)).to(device),
    #torch.zeros(bs, seq_len, dtype=torch.int).to(device),
]
#model_gptj = model_gptj.eval()
#model_gptj.to(device)

# Creating the trace
#traced_model = torch.jit.trace(model_gptj, tokenizer_gptj(prompt, return_tensors="pt").input_ids) #dummy_inputs)
traced_model = torch.jit.script(model_gptj)



Using cpu device


UnsupportedNodeError: GeneratorExp aren't supported:
  File "/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/modeling_utils.py", line 1439
        activations".
        """
        return any(hasattr(m, "gradient_checkpointing") and m.gradient_checkpointing for m in self.modules())
                  ~ <--- HERE


### Use this to save into .PT format -- for Triton server

**If we save using the torch.save() then the model has to be loaded in triton server as Torch.load() and it fails consistently there**

In [50]:
import torch
#model_gptj = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", torch_dtype=torch.float16) 
torch.save(model_gptj, "./triton-serve/bert-gptj/1/model.pt")

**Write the config file**

In [27]:
%%writefile triton-serve/bert-gptj/config.pbtxt
platform: "pytorch_libtorch"
max_batch_size: 32
input [
  {
    name: "INPUT__0"
    data_type: TYPE_INT32
    dims: [512]
  },
  {
    name: "INPUT__1"
    data_type: TYPE_INT32
    dims: [512]
  }
]
output [
  {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [512, 768]
  },
  {
    name: "1634__1"
    data_type: TYPE_FP32
    dims: [768]
  }
]
instance_group {
  count: 1
  kind: KIND_GPU
}
dynamic_batching {
  preferred_batch_size: 32
}

Overwriting triton-serve/bert-gptj/config.pbtxt


### Run for Triton server

**Note**: Amazon SageMaker expects the model tarball file to have a top level directory with the same name as the model defined in the `config.pbtxt`. Below is the sample model directory structure

```
bert-gptj
├── 1
│   └── model.pt
└── config.pbtxt
```

### Create the BERT Model in Torch Script using dummy inputs -- .pt model
Create using the dummy inputs

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))


model = model_gptj #BertModel.from_pretrained("bert-large-uncased", torchscript=True)

bs = 1
seq_len = 512
dummy_inputs = [
    torch.randint(1000, (bs, seq_len)).to(device),
    torch.zeros(bs, seq_len, dtype=torch.int).to(device),
]
model = model.eval()
model.to(device)

traced_model = torch.jit.trace(model, dummy_inputs)
#torch.jit.save(traced_model, "./triton-serve/bert-uc/1/model.pt")

print("Saved {}".format(traced_model))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Using cpu device
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface

IndexError: dimension specified as -2 but tensor has no dimensions

**Predict test using the traced model Needs Tokens and Attention mask both**

In [ ]:
output = traced_model(input_ids=tokens_tensor, attention_mask=segments_tensors)
print(len(output), type(output), type(output[0]))

In [ ]:
output

### UPLOAD of the Model.tar after it has been created correctly by 

Because we share the same model tar with bloom and with bert-uc
rm model.tar.gz in the triton-serve directory

tar --exclude=".git" --exclude=".gitattributes" --exclude="model.tar.gz" --exclude="*.bin" -zcvf model.tar.gz

**Upload the model.tar.gz to S3 location**

In [33]:
import sagemaker
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.utils import name_from_base
import boto3
region = boto3.Session().region_name
role = get_execution_role()
session = sagemaker.Session()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")


In [34]:
s3_model_path_triton = sagemaker.s3.S3Uploader().upload(
    local_path="./triton-serve/model.tar.gz",
    desired_s3_uri="s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bert-gptj",
    sagemaker_session=session
)
s3_mme_model_path='s3://sagemaker-us-east-1-622343165275/bloom/triton_models/'
print(s3_model_path_triton)
print(s3_mme_model_path)

s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bert-gptj/model.tar.gz
s3://sagemaker-us-east-1-622343165275/bloom/triton_models/


#### Start Single Model Triton for starting

**Triton Image download and sagemaker variables**

In [36]:
from sagemaker import get_execution_role, Session, image_uris
import boto3
from sagemaker.utils import name_from_base

region = boto3.Session().region_name
role = get_execution_role()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
triton_image_uri = "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.07-py3".format(
    account_id=account_id_map[region], region=region, base=base
)
print(triton_image_uri)

785573368785.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver:22.07-py3


**Model creation**

In [42]:
endpoint_name_p5 = name_from_base(f"p5-bert-gptj-")
print(endpoint_name_p5)

container_p5 = {
    'Image': triton_image_uri,
    'ModelDataUrl': s3_model_path_triton,
    'Environment': {
        #'SAGEMAKER_PROGRAM' : 'inference.py',
        #'SAGEMAKER_SUBMIT_DIRECTORY' : 'code',
        'SAGEMAKER_TRITON_DEFAULT_MODEL_NAME': 'bert-gptj',
        "SAGEMAKER_TRITON_BATCH_SIZE": "16",
        "SAGEMAKER_TRITON_MAX_BATCH_DELAY": "1000",
        "SAGEMAKER_TRITON_SHM_DEFAULT_BYTE_SIZE" : "16777216", #"16777216000",
        "SAGEMAKER_TRITON_SHM_GROWTH_BYTE_SIZE": "1048576"
    }
}
create_model_response = sm_client.create_model(
    ModelName=endpoint_name_p5, ExecutionRoleArn=role, PrimaryContainer=container_p5
)
print(create_model_response)

p5-bert-gptj--2022-09-13-00-11-32-939
{'ModelArn': 'arn:aws:sagemaker:us-east-1:622343165275:model/p5-bert-gptj--2022-09-13-00-11-32-939', 'ResponseMetadata': {'RequestId': '4e4bab13-15a3-4de3-a1fe-3c37ca5f52fa', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4e4bab13-15a3-4de3-a1fe-3c37ca5f52fa', 'content-type': 'application/x-amz-json-1.1', 'content-length': '99', 'date': 'Tue, 13 Sep 2022 00:11:32 GMT'}, 'RetryAttempts': 0}}


**Endpoint config**

In [44]:
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_name_p5,
    ProductionVariants=[
        {
            "InstanceType": "ml.g4dn.xlarge", #"ml.g4dn.xlarge", ml.g5.8xlarge
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": endpoint_name_p5,
            "VariantName": "AllTraffic",
            #"VolumeSizeInGB": 256,
            "ModelDataDownloadTimeoutInSeconds": 360,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])


Endpoint Config Arn: arn:aws:sagemaker:us-east-1:622343165275:endpoint-config/p5-bert-gptj--2022-09-13-00-11-32-939


**Endpoint**

In [45]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name_p5, EndpointConfigName=endpoint_name_p5
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])


Endpoint Arn: arn:aws:sagemaker:us-east-1:622343165275:endpoint/p5-bert-gptj--2022-09-13-00-11-32-939


In [ ]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5)
status = resp["EndpointStatus"]
print("SINGLE:Model:endpoint:Triton:Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5)
    status = resp["EndpointStatus"]
    print("Single:model:triton:Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Single:model:triton:Status: " + status)

SINGLE:Model:endpoint:Triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: Creating
Single:model:triton:Status: C

**Now Invoke The endpoint**

In [ ]:
#The input prompt is what you give GPT-J. This is limited to 2048 characters (or about 512 tokens) in length. 

In [47]:
import tritonclient.http as httpclient
from transformers import BertTokenizer
import numpy as np


def tokenize_text(text, enc, max_length=512):
    #enc = BertTokenizer.from_pretrained("bert-base-uncased")
    print(f"Tokenize:text:why??::max_length={max_length}::Tokenizer={enc}")
    encoded_text = enc(text, padding="max_length", max_length=max_length)
    return encoded_text["input_ids"], encoded_text["attention_mask"]


def _get_sample_tokenized_text_binary(text, input_names, output_names, enc, max_length=512):
    inputs = []
    outputs = []
    inputs.append(httpclient.InferInput(input_names[0], [1, max_length], "INT32"))
    inputs.append(httpclient.InferInput(input_names[1], [1, max_length], "INT32"))
    indexed_tokens, attention_mask = tokenize_text(text,enc)

    indexed_tokens = np.array(indexed_tokens, dtype=np.int32)
    indexed_tokens = np.expand_dims(indexed_tokens, axis=0)
    inputs[0].set_data_from_numpy(indexed_tokens, binary_data=True)

    attention_mask = np.array(attention_mask, dtype=np.int32)
    attention_mask = np.expand_dims(attention_mask, axis=0)
    inputs[1].set_data_from_numpy(attention_mask, binary_data=True)

    outputs.append(httpclient.InferRequestedOutput(output_names[0], binary_data=True))
    outputs.append(httpclient.InferRequestedOutput(output_names[1], binary_data=True))
    request_body, header_length = httpclient.InferenceServerClient.generate_request_body(
        inputs, outputs=outputs
    )
    return request_body, header_length


def get_sample_tokenized_text_binary_pt(text, enc, max_length=512):
    return _get_sample_tokenized_text_binary(
        text, ["INPUT__0", "INPUT__1"], ["OUTPUT__0", "1634__1"], enc, max_length
    )


def get_sample_tokenized_text_binary_trt(text, enc):
    return _get_sample_tokenized_text_binary(text, ["token_ids", "attn_mask"], ["output", "1634"], enc, max_length)

In [48]:
%%time

import json
max_seq_length=512
text_triton = "Triton Inference Server provides a cloud and edge inferencing solution optimized for both CPUs and GPUs."
print(f"Leverage the Tokenizer={tokenizer_gptj}::max_seq_length={max_seq_length}:: create above when creating the model ")
input_ids, attention_mask = tokenize_text(text_triton, tokenizer_gptj, max_length=max_seq_length)

payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids},
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_p5, ContentType="application/octet-stream", Body=json.dumps(payload)
)

print(json.loads(response["Body"].read().decode("utf8")))

Using pad_token, but it is not set yet.


Leverage the Tokenizer=PreTrainedTokenizer(name_or_path='EleutherAI/gpt-j-6B', vocab_size=50257, model_max_len=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)})::max_seq_length=512:: create above when creating the model 
Tokenize:text:why??::max_length=512::Tokenizer=PreTrainedTokenizer(name_or_path='EleutherAI/gpt-j-6B', vocab_size=50257, model_max_len=2048, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=Tr

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
endpoint_name_p5

In [ ]:
%%time
import json

max_seq_length=512
text_triton = """This is a creative writing exercise. Below, you'll be given a prompt. Your story should be based on the prompt.

Prompt: A scary story about a haunted mouse
Story: On a dark and stormy night, the mouse crept in the shadows. """


print(f"Leverage the Tokenizer={enc}::max_seq_length={max_seq_length}:: create above when creating the model ")
input_ids, attention_mask = tokenize_text(text_triton, enc, max_length=max_seq_length)

payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids},
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}


response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_p5, ContentType="application/octet-stream", Body=json.dumps(payload)
)

output_dict = json.loads(response["Body"].read().decode("utf8"))

# -- output_dict['outputs'][0]['data']  -- has 0 and 1 as 2 indexes in list 
output_dict.keys()

#enc.decode(output_dict['outputs'][0]['data'], skip_special_tokens=True)

In [ ]:
input_ids
attention_mask 

# open file in write mode
with open(r'./temp-bloom/input_ids.txt', 'w') as fp:
    for item in input_ids:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done input_ids')
    
# open file in write mode
with open(r'./temp-bloom/attention_mask.txt', 'w') as fp:
    for item in attention_mask:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done attention_mask')


### Clean up

In [51]:
sm_client.delete_endpoint(EndpointName=endpoint_name_p5)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name_p5)
sm_client.delete_model(ModelName=endpoint_name_p5)

{'ResponseMetadata': {'RequestId': '522e6743-b93c-464a-8721-2a125a25ff64',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '522e6743-b93c-464a-8721-2a125a25ff64',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 13 Sep 2022 16:52:24 GMT'},
  'RetryAttempts': 0}}

In [ ]:
# general imports
import boto3
import json
import os
import re
import copy
import time
from time import gmtime, strftime
import numpy as np
import datetime
import pprint
import pandas as pd

# sagemaker
import sagemaker
from sagemaker import get_execution_role

# triton
import tritonclient.http as httpclient

# transformers
from transformers import BertTokenizer

# custom CloudWatch
#from cloudwatch import get_endpoint_metrics


In [ ]:
!docker run --gpus=all --rm -it  -v `pwd`/workspace:/workspace nvcr.io/nvidia/pytorch:21.08-py3 /bin/bash generate_models.sh

## START MME for triton 

**Upload first**

### Upload multiple copies for MME

In [ ]:
for ii in range(1,100):
    s3_model_path_triton_mme = sagemaker.s3.S3Uploader().upload(
        local_path="./triton-serve/model.tar.gz",
        desired_s3_uri=f"s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bert-gptj/model-{ii}",
        sagemaker_session=session
    )
s3_model_path_mme='s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bert-gptj'
print("MULTIPLE:Uploaded")
print(s3_model_path_triton_mme)
print(s3_model_path_mme)

In [ ]:
model

In [ ]:
s3_model_path_mme

**Create the model**

In [ ]:
endpoint_name_p5_mme = name_from_base(f"p5-bert-uc-mme")
print(endpoint_name_p5_mme)

container_p5_mme = {
    'Image': triton_image_uri,
    'ModelDataUrl': s3_model_path_mme,
    'Mode':'MultiModel',
    'Environment': {
        #'SAGEMAKER_PROGRAM' : 'inference.py',
        #'SAGEMAKER_SUBMIT_DIRECTORY' : 'code',
        'SAGEMAKER_TRITON_DEFAULT_MODEL_NAME': 'model-1',
        "SAGEMAKER_TRITON_BATCH_SIZE": "16",
        "SAGEMAKER_TRITON_MAX_BATCH_DELAY": "1000",
        "SAGEMAKER_TRITON_SHM_DEFAULT_BYTE_SIZE" : "16777216", #"16777216000",
        "SAGEMAKER_TRITON_SHM_GROWTH_BYTE_SIZE": "1048576"
    }
}
create_model_response_mme = sm_client.create_model(
    ModelName=endpoint_name_p5_mme, ExecutionRoleArn=role, PrimaryContainer=container_p5_mme
)
print(create_model_response_mme)

**Create the Endpoint config**

In [ ]:
create_endpoint_config_response_mme = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_name_p5_mme,
    ProductionVariants=[
        {
            "InstanceType": "ml.g4dn.xlarge", #"ml.g4dn.xlarge",ml.g5.8xlarge
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": endpoint_name_p5_mme,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response_mme["EndpointConfigArn"])

**Create the endpoint**

In [ ]:
create_endpoint_response_mme = sm_client.create_endpoint(
    EndpointName=endpoint_name_p5_mme, EndpointConfigName=endpoint_name_p5_mme
)

print("Endpoint Arn: " + create_endpoint_response_mme["EndpointArn"])


In [ ]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5_mme)
status = resp["EndpointStatus"]
print("MME:Model:endpoint:Triton:Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5_mme)
    status = resp["EndpointStatus"]
    print("MME:model:triton:Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("MME:model:triton:Status: " + status)

**Test the end point**

In [ ]:
%%time

import json
max_seq_length=512
text_triton = "Triton Inference Server provides a cloud and edge inferencing solution optimized for both CPUs and GPUs."
print(f"Leverage the Tokenizer={enc}::max_seq_length={max_seq_length}:: create above when creating the model ")
input_ids, attention_mask = tokenize_text(text_triton, enc, max_length=max_seq_length)

payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids},
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_p5_mme, ContentType="application/octet-stream", Body=json.dumps(payload), TargetModel  = "/model-9/model.tar.gz"
)

output_dict = json.loads(response["Body"].read().decode("utf8"))

# -- output_dict['outputs'][0]['data']  -- has 0 and 1 as 2 indexes in list 
output_dict.keys()

enc.decode(output_dict['outputs'][0]['data'], skip_special_tokens=True)

In [ ]:
endpoint_name_p5_mme

In [ ]:
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name_p5_mme, ContentType="text/json", Body=json.dumps(payload), TargetModel  = "/model-9/model.tar.gz"
)
output_dict = json.loads(response["Body"].read().decode("utf8"))

# -- output_dict['outputs'][0]['data']  -- has 0 and 1 as 2 indexes in list 
output_dict.keys()

**set up in S3 payload to be used for inference load testing**

In [ ]:
max_seq_length=512
text_triton = """
                Create payload JSON and upload it on S3. 
                This will be used by Inference Recommender to run the load test.
              """

input_ids, attention_mask = tokenize_text(text_triton, enc, max_length=max_seq_length)

payload = {
    "inputs": [
        {"name": "INPUT__0", "shape": [1, max_seq_length], "datatype": "INT32", "data": input_ids},
        {"name": "INPUT__1", "shape": [1, max_seq_length], "datatype": "INT32", "data": attention_mask},
    ]
}

print(f"Sample payload to be used with Inference Recommender")
print(payload)

payload_location = "./sample-payload/"
!mkdir -p $payload_location

payload_archive_name = "payload.tar.gz"

with open(payload_location + "request.json", "w") as f:
    json.dump(payload, f)


!cd ./sample-payload/ && tar czvf ../payload.tar.gz *

print(f"payload.tar.gz created at {payload_location}/{payload_archive_name}")

**Upload sample payload to S3**

In [ ]:
s3_sample_data_path_triton = sagemaker.s3.S3Uploader().upload(
    local_path=f"{payload_archive_name}",
    desired_s3_uri="s3://sagemaker-us-east-1-622343165275/bloom/triton_test_data",
    sagemaker_session=session
)
s3_sample_data_path_triton

## Inference Load test set up
### DOES NOT WORK FOR MME -- SO SKIP this section

In [ ]:
ml_domain = "NATURAL_LANGUAGE_PROCESSING"
ml_task = "FILL_MASK"
ml_framework = "PYTORCH"
framework_version = "1.6.0"
model_tested = "bert-base-uncased"

In [ ]:
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sm_model_name = "pt-triton-benchmark-model-" + ts
model_package_group_name = "pt-triton-benchmark-model-group-" + ts
advanced_job = "pt-triton-benchmark-advanced-job-" + ts

print(f"SageMaker Model Name: {sm_model_name}")
print(f"SageMaker Mode Package Name: {model_package_group_name}")
print(f"SageMaker Advanced Job Name: {advanced_job}")

In [ ]:
s3_model_path_mme

In [ ]:
container_infrec_mme = {
    'Image': triton_image_uri,
    "NearestModelName": model_tested, #'model-1',
    "Framework": ml_framework,
    'ModelDataUrl': s3_model_path_mme,
    #'Mode':'MultiModel',
    'Environment': {
        #'SAGEMAKER_PROGRAM' : 'inference.py',
        #'SAGEMAKER_SUBMIT_DIRECTORY' : 'code',
        'SAGEMAKER_TRITON_DEFAULT_MODEL_NAME': 'model-1',
        "SAGEMAKER_TRITON_BATCH_SIZE": "16",
        "SAGEMAKER_TRITON_MAX_BATCH_DELAY": "1000",
        "SAGEMAKER_TRITON_SHM_DEFAULT_BYTE_SIZE" : "16777216", #"16777216000",
        "SAGEMAKER_TRITON_SHM_GROWTH_BYTE_SIZE": "1048576"
    },
}


In [ ]:
model_pacakge_group_response = sm_client.create_model_package_group(
    ModelPackageGroupName=str(model_package_group_name),
    ModelPackageGroupDescription="BERT large uncased Model group for Triton Serving",
)
print(f"Model Registry package group: {model_pacakge_group_response}")


In [ ]:
model_package_version_response = sm_client.create_model_package(
    ModelPackageGroupName=str(model_package_group_name),
    ModelPackageDescription="BERT large uncased Model group for Triton Serving",
    Domain=ml_domain,
    Task=ml_task,
    SamplePayloadUrl=s3_sample_data_path_triton,
    InferenceSpecification={
        "Containers": [container_infrec_mme],
        "SupportedRealtimeInferenceInstanceTypes": [
            "ml.g4dn.4xlarge",
            "ml.g4dn.4xlarge",
        ],
        "SupportedContentTypes": ["application/octet-stream"],
        "SupportedResponseMIMETypes": ["application/json"],
    },
)
model_package_version_response

In [ ]:
advanced_response = sm_client.create_inference_recommendations_job(
    JobName=advanced_job,
    JobDescription="nlp triton Inference Advanced Recommender Job",
    JobType="Advanced",
    RoleArn=role,
    InputConfig={
        "ModelPackageVersionArn": model_package_version_response["ModelPackageArn"],
        "JobDurationInSeconds": 7200,
        "EndpointConfigurations": [
            #{"InstanceType": "ml.p3.8xlarge"},
            #{"InstanceType": "ml.p3.2xlarge"},
            {"InstanceType": "ml.p2.16xlarge"},
            {"InstanceType": "ml.g4dn.xlarge"},
            {"InstanceType": "ml.g4dn.8xlarge"},
            {"InstanceType": "ml.g4dn.4xlarge"},
            {"InstanceType": "ml.g4dn.2xlarge"},
            {"InstanceType": "ml.g4dn.12xlarge"},
        ],
        "TrafficPattern": {
            "TrafficType": "PHASES",
            "Phases": [
                {
                    "InitialNumberOfUsers": 2,
                    "SpawnRate": 3,
                    "DurationInSeconds": 900,
                },  # simulating 50 users, 2 initial and 3 new users every minute for 16 minutes
            ],  # second phase, we will strt with 50 users, steady traffic for 5 minutes
        },
        "ResourceLimit": {"MaxNumberOfTests": 10, "MaxParallelOfTests": 5},
    },
    StoppingConditions={
        "MaxInvocations": 30000,
        "ModelLatencyThresholds": [{"Percentile": "P95", "ValueInMilliseconds": 500}],
    },
)

print(advanced_response)


In [ ]:
%%time

ended = False
while not ended:
    inference_recommender_job = sm_client.describe_inference_recommendations_job(
        JobName=str(advanced_job)
    )
    if inference_recommender_job["Status"] in ["COMPLETED", "STOPPED", "FAILED"]:
        print(f"Inference recommender job status: {inference_recommender_job['Status']} ")
        ended = True
    else:
        print("Inference recommender job in progress")
        time.sleep(300)

if inference_recommender_job["Status"] == "FAILED":
    print("Inference recommender job failed ")
    print("Failed Reason: {}".inference_recommender_job["FailedReason"])
else:
    print("Inference recommender job completed")

## Clean up

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name_p5_mme)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_name_p5_mme)
sm_client.delete_model(ModelName=endpoint_name_p5_mme)